In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from peft import LoraConfig

dataset = load_dataset("lucasmccabe-lmi/CodeAlpaca-20k", split="train")

model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")

def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['instruction'])):
        text = f"### Question: {example['instruction'][i]}\n ### Answer: {example['output'][i]}"
        output_texts.append(text)
    return output_texts

response_template = " ### Answer:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)


TypeError: OPTForCausalLM.__init__() got an unexpected keyword argument 'device'

In [8]:
from peft import LoraConfig, get_peft_model
from peft import prepare_model_for_kbit_training

from transformers import TrainingArguments


model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

#Lora config
LORA_R = 4
LORA_ALPHA = 16
LORA_DROPOUT = 0.05

peft_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)

OUTPUT_DIR = 'test-finetuned-opt-350m'
# OPTIM = "paged_adamw_32bit"


# Settings for T4 (A100)
MICRO_BATCH_SIZE = 2 # 4? for A100
BATCH_SIZE = 128
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE # or just 4
EPOCHS = 1
LEARNING_RATE = 2e-4 #2e-5
CUTOFF_LEN = 256

WARMUP_RATIO = 0.03
MAX_STEPS = 100 # 300
MAX_GRAD_NORM = 0.3
LR_SCHEDULER_TYPE = "constant"
SAVE_STEPS = 50
LOGGING_STEPS = 5


training_args = TrainingArguments(
    per_device_train_batch_size=MICRO_BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    num_train_epochs=EPOCHS,
    # optim=OPTIM,
    warmup_ratio=WARMUP_RATIO, # warmup_steps=100,
    learning_rate=LEARNING_RATE,
    lr_scheduler_type=LR_SCHEDULER_TYPE,
    # fp16=True,
    # evaluation_strategy = "steps",
    save_steps= SAVE_STEPS, # save_total_limit=3,
    output_dir=OUTPUT_DIR,
    report_to='tensorboard',
    logging_steps=LOGGING_STEPS,
    logging_dir=OUTPUT_DIR+'/logs',
    # load_best_model_at_end=True, # requires evaluation strategy and save strategy to match
)


trainer = SFTTrainer(
    model,
    train_dataset=dataset,
    args=training_args,
    max_seq_length=256,
    formatting_func=formatting_prompts_func,
    data_collator=collator,
    peft_config=peft_config,
)

trainer.train() 
trainer.save_model('test-finetuned-opt-350m')

Map:   0%|          | 0/20022 [00:00<?, ? examples/s]

  0%|          | 0/156 [00:00<?, ?it/s]

python(44094) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(44095) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(44096) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(44097) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(44098) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(44099) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(44100) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(44101) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(44102) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(44104) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(44105) Malloc

{'loss': 2.321, 'learning_rate': 0.0002, 'epoch': 0.03}


python(50035) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(50036) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(50037) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(50038) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(50039) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(50040) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(50045) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(50046) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(50049) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(50055) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(50059) Malloc

{'loss': 2.2014, 'learning_rate': 0.0002, 'epoch': 0.06}


python(57441) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(57442) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(57443) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(57445) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(57447) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(57448) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(57449) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(57450) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(57455) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(57456) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(57457) Malloc

{'loss': 2.116, 'learning_rate': 0.0002, 'epoch': 0.1}


python(65077) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(65081) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(65082) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(65084) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(65085) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(65090) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(65097) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(65099) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(65100) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(65101) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(65102) Malloc

KeyboardInterrupt: 

In [ ]:
# trainer.save_model('test-finetuned-opt-350m')